<a href="https://colab.research.google.com/github/RodrigoEduard/Codigos-TFM-VIU-2023/blob/Procesamiento_y_Clasificacion_de_Datos_por_Zonas/TFM_VIU_2023_22_Procesamiento_Clasificaci%C3%B3n_por_Zonas_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-------------------------------------------------------------------------------
#----- DISTRIBUCIÓN DE REGIONES POR CADA UNA DE LAS ZONAS
#-------------------------------------------------------------------------------
# ZONA 1 (NORTE): XV, I, II, III, IV: 24 comunas con juzgados
#   DE ARICA Y PARINACOTA(1), DE TARAPACA(3), DE ANTOFAGASTA(6), DE ATACAMA(6), DE COQUIMBO(8)
#-------------------------------------------------------------------------------
# ZONA 2: V, RM (CENTRO METROPOLITANA): 24 comunas
#   DE VALPARAISO(16), METROPOLITANA DE SANTIAGO(8))
#-------------------------------------------------------------------------------
# ZONA 3: VI, VII, VII, XVI, VIII (CENTRO SUR): 47 comunas
#   DEL LIBERTADOR B. O'HIGGINS(10), DEL MAULE(11), DE ÑUBLE(6), DEL BIOBIO(20)
#-------------------------------------------------------------------------------
# ZONA 4: IX, XIV, X (SUR): 35 comunas
#   DE LA ARAUCANIA(15), DE LOS RIOS(7), DE LOS LAGOS(13)
#-------------------------------------------------------------------------------
# ZONA 5 (AUSTRAL): XI y XII: 9
#   DE AISEN DEL GRAL. C. IBAÑEZ DEL CAMPO(5), DE MAGALLANES Y DE LA ANTARTICA CHILENA(4)
#-------------------------------------------------------------------------------

library(readxl)
zonas <- read_excel("conduccion_130823_2015_2023.xlsx")
zonas$`MES INGRESO` <- as.numeric(zonas$`MES INGRESO`)
# Crear una nueva columna 'FECHA' combinando 'AÑO INGRESO' y 'MES INGRESO' en el formato "aYY.MM"
zonas$FECHA <- paste('a', substr(zonas$`AÑO INGRESO`, 3, 4), '.', sprintf("%02d", zonas$`MES INGRESO`), sep = "")
zonas$FECHA <- paste(zonas$`AÑO INGRESO`, sprintf("%02d", zonas$`MES INGRESO`), sep = ".")

# ZONA NORTE: XV, I, II, III, IV: 24 comunas
# DE ARICA Y PARINACOTA(1), DE TARAPACA(3), DE ANTOFAGASTA(6), DE ATACAMA(6), DE COQUIMBO(8)
regiones <- c('DE ARICA Y PARINACOTA', 'DE TARAPACA', 'DE ANTOFAGASTA','DE ATACAMA', 'DE COQUIMBO')
dat <- subset(zonas, `NOMBRE REGION` %in% regiones)
dat = table(dat$`NOMBRE REGION`,dat$`FECHA`)
dat = as.data.frame(dat)
dat = table(dat$`NOMBRE COMUNA`,dat$`FECHA`)

pde_f = 100*(dat/rowSums(dat))
pde_f = as.data.frame(pde_f)
max = max(pde_f[,3],na.rm=T)
max = round(max,digits = 0)
mediana = median(pde_f[,3],na.rm=T)  #calcula la mediana de la tercera columna del dataframe pde_f, ignorando cualquier valor NA, y luego asigna ese valor a la variable mediana.
pde_f$clase = 0
pde_f$clase[pde_f$Freq == mediana] <- 'igual'
pde_f$clase[pde_f$Freq > mediana] <- 'sobre'
pde_f$clase[pde_f$Freq < mediana] <- 'bajo'
library(tidyverse)
ZNO1 <- pde_f %>%
  rename(Comuna = Var1) %>%
  group_by(Comuna, Var2) %>%
  summarise(clase = first(clase), .groups = "drop") %>%
  pivot_wider(names_from = Var2, values_from = clase)

nombres_columnas <- colnames(ZNO1)[-1]  # Se toma los nombres de las columnas excepto "Comuna"
partes <- strsplit(nombres_columnas, "\\.") # Separé en años y meses usando el delimitador "."
años <- sapply(partes, `[`, 1)
meses <- sapply(partes, `[`, 2)
meses_abreviados <- months(as.Date(paste0(años, "-", meses, "-01")), abbreviate = TRUE) # Convertir los números de meses a abreviaturas de meses
# Combinando las abreviaturas de los meses con los años
nuevos_nombres_columnas <- c("Comuna", paste0(toupper(substr(meses_abreviados, 1, 1)), tolower(substr(meses_abreviados, 2, 3)), ".", substr(años, 3, 4)))
colnames(ZNO1) <- nuevos_nombres_columnas # Asignando los nuevos nombres de columnas a ZNO1

ZNO1 <- ZNO1 %>%
  select(Comuna, one_of(nuevos_nombres_columnas))
#TRATAMIENTO ESPECIAL PARA LAS COMUNAS SIN JUZGADOS (PERIODO: ENE.2015 A NOV.2016)
comunas_ajustar <- c("ALTO HOSPICIO", "MEJILLONES")
meses_ajustar <- c(paste0(toupper(substr(months(as.Date(paste0("20", rep(15, times = 12), "-", sprintf("%02d", 1:12), "-01")), abbreviate = TRUE), 1, 1)),
                          tolower(substr(months(as.Date(paste0("20", rep(15, times = 12), "-", sprintf("%02d", 1:12), "-01")), abbreviate = TRUE), 2, 3)), ".15"),
                   paste0(toupper(substr(months(as.Date(paste0("20", rep(16, times = 10), "-", sprintf("%02d", 1:10), "-01")), abbreviate = TRUE), 1, 1)),
                          tolower(substr(months(as.Date(paste0("20", rep(16, times = 10), "-", sprintf("%02d", 1:10), "-01")), abbreviate = TRUE), 2, 3)), ".16"))
ZNO1 <- as.data.frame(ZNO1)
for (comuna in comunas_ajustar) {
  for (mes in meses_ajustar) {
    ZNO1[ZNO1$Comuna == comuna, mes] <- "sin_juzgado"
  }
}
library(writexl)
write_xlsx(ZNO1,"01-zonaNorte.xlsx")

# ZONA 2: V, RM (CENTRO-METROPOLITANA): 24 comunas
#   DE VALPARAISO(16), METROPOLITANA DE SANTIAGO(8))
regiones <- c('DE VALPARAISO','METROPOLITANA DE SANTIAGO')
dat <- subset(zonas, `NOMBRE REGION` %in% regiones)
dat = table(dat$`NOMBRE COMUNA`,dat$`FECHA`)

pde_f = 100*(dat/rowSums(dat))
pde_f = as.data.frame(pde_f)
max = max(pde_f[,3],na.rm=T)
max = round(max,digits = 0)
mediana = median(pde_f[,3],na.rm=T)  #calcula la mediana de la tercera columna del dataframe pde_f, ignorando cualquier valor NA, y luego asigna ese valor a la variable mediana.
pde_f$clase = 0
pde_f$clase[pde_f$Freq == mediana] <- 'igual'
pde_f$clase[pde_f$Freq > mediana] <- 'sobre'
pde_f$clase[pde_f$Freq < mediana] <- 'bajo'

library(tidyverse)
ZCM2 <- pde_f %>%
  rename(id = Var1) %>%
  group_by(id, Var2) %>%
  summarise(clase = first(clase), .groups = "drop") %>%
  pivot_wider(names_from = Var2, values_from = clase)

nombres_columnas <- colnames(ZCM2)[-1]  # Se toma los nombres de las columnas excepto "Comuna"
partes <- strsplit(nombres_columnas, "\\.") # Separé en años y meses usando el delimitador "."
años <- sapply(partes, `[`, 1)
meses <- sapply(partes, `[`, 2)
meses_abreviados <- months(as.Date(paste0(años, "-", meses, "-01")), abbreviate = TRUE) # Convertir los números de meses a abreviaturas de meses
# Combinando las abreviaturas de los meses con los años
nuevos_nombres_columnas <- c("Comuna", paste0(toupper(substr(meses_abreviados, 1, 1)), tolower(substr(meses_abreviados, 2, 3)), ".", substr(años, 3, 4)))
colnames(ZCM2) <- nuevos_nombres_columnas # Asignando los nuevos nombres de columnas a ZCM2

ZCM2 <- ZCM2 %>%
  select(Comuna, one_of(nuevos_nombres_columnas))

library(writexl)
write_xlsx(ZCM2,"02-zonaCentroMetropolitana.xlsx")

# ZONA 3: VI, VII, XVI, VIII (CENTRO): 47 comunas
#   DEL LIBERTADOR B. O'HIGGINS(10), DEL MAULE(11), DE ÑUBLE(6), DEL BIOBIO(20)
regiones <- c("DEL LIBERTADOR B. O'HIGGINS",'DEL MAULE','DE ÑUBLE','DEL BIOBIO')
dat <- subset(zonas, `NOMBRE REGION` %in% regiones)
dat = table(dat$`NOMBRE COMUNA`,dat$`FECHA`)

pde_f = 100*(dat/rowSums(dat))
pde_f = as.data.frame(pde_f)
max = max(pde_f[,3],na.rm=T)
max = round(max,digits = 0)
mediana = median(pde_f[,3],na.rm=T)  #calcula la mediana de la tercera columna del dataframe pde_f, ignorando cualquier valor NA, y luego asigna ese valor a la variable mediana.
pde_f$clase = 0
pde_f$clase[pde_f$Freq == mediana] <- 'igual'
pde_f$clase[pde_f$Freq > mediana] <- 'sobre'
pde_f$clase[pde_f$Freq < mediana] <- 'bajo'

library(tidyverse)
ZCE3 <- pde_f %>%
  rename(id = Var1) %>%
  group_by(id, Var2) %>%
  summarise(clase = first(clase), .groups = "drop") %>%
  pivot_wider(names_from = Var2, values_from = clase)

nombres_columnas <- colnames(ZCE3)[-1]  # Se toma los nombres de las columnas excepto "Comuna"
partes <- strsplit(nombres_columnas, "\\.") # Separé en años y meses usando el delimitador "."
años <- sapply(partes, `[`, 1)
meses <- sapply(partes, `[`, 2)
meses_abreviados <- months(as.Date(paste0(años, "-", meses, "-01")), abbreviate = TRUE) # Convertir los números de meses a abreviaturas de meses
# Combinando las abreviaturas de los meses con los años
nuevos_nombres_columnas <- c("Comuna", paste0(toupper(substr(meses_abreviados, 1, 1)), tolower(substr(meses_abreviados, 2, 3)), ".", substr(años, 3, 4)))
colnames(ZCE3) <- nuevos_nombres_columnas # Asignando los nuevos nombres de columnas a ZCE3

ZCE3 <- ZCE3 %>%
  select(Comuna, one_of(nuevos_nombres_columnas))
library(writexl)
write_xlsx(ZCE3,"03-ZonaCentro.xlsx")

# ZONA 4: IX, XIV, X (SUR): 35 comunas
#   DE LA ARAUCANIA(15), DE LOS RIOS(7), DE LOS LAGOS(13)
regiones <- c("DE LA ARAUCANIA",'DE LOS RIOS','DE LOS LAGOS')
dat <- subset(zonas, `NOMBRE REGION` %in% regiones)
dat = table(dat$`NOMBRE COMUNA`,dat$`FECHA`)

pde_f = 100*(dat/rowSums(dat))
pde_f = as.data.frame(pde_f)
max = max(pde_f[,3],na.rm=T)
max = round(max,digits = 0)
mediana = median(pde_f[,3],na.rm=T)  #calcula la mediana de la tercera columna del dataframe pde_f, ignorando cualquier valor NA, y luego asigna ese valor a la variable mediana.
pde_f$clase = 0
pde_f$clase[pde_f$Freq == mediana] <- 'igual'
pde_f$clase[pde_f$Freq > mediana] <- 'sobre'
pde_f$clase[pde_f$Freq < mediana] <- 'bajo'

library(tidyverse)
ZSU4 <- pde_f %>%
  rename(id = Var1) %>%
  group_by(id, Var2) %>%
  summarise(clase = first(clase), .groups = "drop") %>%
  pivot_wider(names_from = Var2, values_from = clase)

nombres_columnas <- colnames(ZSU4)[-1]  # Se toma los nombres de las columnas excepto "Comuna"
partes <- strsplit(nombres_columnas, "\\.") # Separé en años y meses usando el delimitador "."
años <- sapply(partes, `[`, 1)
meses <- sapply(partes, `[`, 2)
meses_abreviados <- months(as.Date(paste0(años, "-", meses, "-01")), abbreviate = TRUE) # Convertir los números de meses a abreviaturas de meses
# Combinando las abreviaturas de los meses con los años
nuevos_nombres_columnas <- c("Comuna", paste0(toupper(substr(meses_abreviados, 1, 1)), tolower(substr(meses_abreviados, 2, 3)), ".", substr(años, 3, 4)))
colnames(ZSU4) <- nuevos_nombres_columnas # Asignando los nuevos nombres de columnas a ZSU4

ZSU4 <- ZSU4 %>%
  select(Comuna, one_of(nuevos_nombres_columnas))

library(writexl)
write_xlsx(ZSU4,"04-ZonaSur.xlsx")

# ZONA 5 (SUR-AUSTRAL): XI y XII: 9 comunas
#   DE AISEN DEL GRAL. C. IBAÑEZ DEL CAMPO(5), DE MAGALLANES Y DE LA ANTARTICA CHILENA(4)
regiones <- c('DE AISEN DEL GRAL. C. IBAÑEZ DEL CAMPO','DE MAGALLANES Y DE LA ANTARTICA CHILENA')
dat <- subset(zonas, `NOMBRE REGION` %in% regiones)
dat = table(dat$`NOMBRE COMUNA`,dat$`FECHA`)

pde_f = 100*(dat/rowSums(dat))
pde_f = as.data.frame(pde_f)
max = max(pde_f[,3],na.rm=T)
max = round(max,digits = 0)
mediana = median(pde_f[,3],na.rm=T)  #calcula la mediana de la tercera columna del dataframe pde_f, ignorando cualquier valor NA, y luego asigna ese valor a la variable mediana.
pde_f$clase = 0
pde_f$clase[pde_f$Freq == mediana] <- 'igual'
pde_f$clase[pde_f$Freq > mediana] <- 'sobre'
pde_f$clase[pde_f$Freq < mediana] <- 'bajo'

library(tidyverse)
ZSA5 <- pde_f %>%
  rename(id = Var1) %>%
  group_by(id, Var2) %>%
  summarise(clase = first(clase), .groups = "drop") %>%
  pivot_wider(names_from = Var2, values_from = clase)

nombres_columnas <- colnames(ZSA5)[-1]  # Se toma los nombres de las columnas excepto "Comuna"
partes <- strsplit(nombres_columnas, "\\.") # Separé en años y meses usando el delimitador "."
años <- sapply(partes, `[`, 1)
meses <- sapply(partes, `[`, 2)
meses_abreviados <- months(as.Date(paste0(años, "-", meses, "-01")), abbreviate = TRUE) # Convertir los números de meses a abreviaturas de meses
# Combinando las abreviaturas de los meses con los años
nuevos_nombres_columnas <- c("Comuna", paste0(toupper(substr(meses_abreviados, 1, 1)), tolower(substr(meses_abreviados, 2, 3)), ".", substr(años, 3, 4)))
colnames(ZSA5) <- nuevos_nombres_columnas # Asignando los nuevos nombres de columnas a ZSA5

ZSA5 <- ZSA5 %>%
  select(Comuna, one_of(nuevos_nombres_columnas))
#TRATAMIENTO ESPECIAL PARA LA COMUNA SIN JUZGADOS (PERIODO: ENE.2015 A NOV.2016)
comunas_ajustar <- c("CABO HORNOS")
meses_ajustar <- c(paste0(toupper(substr(months(as.Date(paste0("20", rep(15, times = 12), "-", sprintf("%02d", 1:12), "-01")), abbreviate = TRUE), 1, 1)),
                          tolower(substr(months(as.Date(paste0("20", rep(15, times = 12), "-", sprintf("%02d", 1:12), "-01")), abbreviate = TRUE), 2, 3)), ".15"),
                   paste0(toupper(substr(months(as.Date(paste0("20", rep(16, times = 10), "-", sprintf("%02d", 1:10), "-01")), abbreviate = TRUE), 1, 1)),
                          tolower(substr(months(as.Date(paste0("20", rep(16, times = 10), "-", sprintf("%02d", 1:10), "-01")), abbreviate = TRUE), 2, 3)), ".16"))
ZSA5 <- as.data.frame(ZSA5)
for (comuna in comunas_ajustar) {
  for (mes in meses_ajustar) {
    ZSA5[ZSA5$Comuna == comuna, mes] <- "sin_juzgado"
  }
}

library(writexl)
write_xlsx(ZSA5,"05-ZonaSurAustral.xlsx")


# TODO CHILE
NACIONAL <-  rbind(ZNO1,ZCM2,ZCE3,ZSU4,ZSA5)
library(writexl)
write_xlsx(NACIONAL,"06-nacional.xlsx")